In [1]:
import pandas
from janome.tokenizer import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

In [2]:
# データセット自体の再配布は禁止なので、Gitコミット時に出力を消す
def no_redistribution(data):
    pass
    # return data

## 学習コードの実装

In [3]:
def load_dataset(dataset_tsv_path):
    df = df = pandas.read_table(dataset_tsv_path)
    df = df.dropna(subset=['label'])
    print('length: ', len(df))
    return df['text'].values, df['label'].values

In [4]:
no_redistribution(load_dataset('/app/data/purin.tsv'))

length:  4125


In [5]:
t = Tokenizer()
def tokenize(text):
    return t.tokenize(text, wakati=True)

list(tokenize("今日のご飯は焼肉です"))

['今日', 'の', 'ご飯', 'は', '焼肉', 'です']

In [6]:
from numpy import count_nonzero

class Trainer():
    def __init__(self, x_train, y_train, x_test, y_test,
        lowercase=False, tokenize=None, preprocessor=None) -> None:
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.vectorizer = CountVectorizer(
            lowercase=lowercase,
            tokenizer=tokenize,
            preprocessor=preprocessor)

    def train(self):
        x_train_vec = self.vectorizer.fit_transform(self.x_train)
        # 不均衡データセットなので重みをつける
        weights = {
            0: 1 / count_nonzero(y_train == 0),
            1: 1 / count_nonzero(y_train == 1),
        }
        self.model = LogisticRegression(solver='liblinear', class_weight=weights)
        self.model.fit(x_train_vec, self.y_train)
        return self.model

    def predict(self, x, proba = True):
        x_vec = self.vectorizer.transform(x)
        if proba:
            predictor = self.model.predict_proba
        else:
            predictor = self.model.predict
        return predictor(x_vec)

    def eval_and_print(self):
        y_pred = self.predict(self.x_test, proba=False)
        print('roc_auc_score (macro average)', roc_auc_score(self.y_test, y_pred, average='macro'))
        tn, fp, fn, tp = confusion_matrix(self.y_test, y_pred).ravel()
        print('tn, fp, fn, tp', tn, fp, fn, tp)
        print(classification_report(self.y_test, y_pred))
    
    def trailn_and_eval(self):
        self.train()
        self.eval_and_print()


In [7]:
x, y = load_dataset('/app/data/purin.tsv')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
trainer = Trainer(x_train, y_train, x_test, y_test, tokenize=tokenize)

model = trainer.train()

length:  4125


In [8]:
trainer.eval_and_print()

roc_auc_score (macro average) 0.7871838042049935
tn, fp, fn, tp 54 10 205 556
              precision    recall  f1-score   support

           0       0.21      0.84      0.33        64
           1       0.98      0.73      0.84       761

    accuracy                           0.74       825
   macro avg       0.60      0.79      0.59       825
weighted avg       0.92      0.74      0.80       825



In [9]:
trainer.predict(['ポケモンのプリンとニャースとイーブイかわいい'])

array([[0.46503863, 0.53496137]])

In [10]:
trainer.predict(['セブンのスイーツの焼きプリンおいしい'])

array([[0.5058649, 0.4941351]])

## 前処理の比較

In [11]:
Trainer(x_train, y_train, x_test, y_test, tokenize=tokenize).trailn_and_eval()


roc_auc_score (macro average) 0.7871838042049935
tn, fp, fn, tp 54 10 205 556
              precision    recall  f1-score   support

           0       0.21      0.84      0.33        64
           1       0.98      0.73      0.84       761

    accuracy                           0.74       825
   macro avg       0.60      0.79      0.59       825
weighted avg       0.92      0.74      0.80       825



In [12]:
import re
def remove_anchor(text):
    return re.sub(r'\@[a-zA-Z0-9]+', '', text)

Trainer(x_train, y_train, x_test, y_test, tokenize=tokenize, preprocessor=remove_anchor).trailn_and_eval()


roc_auc_score (macro average) 0.7977681504599212
tn, fp, fn, tp 53 11 177 584
              precision    recall  f1-score   support

           0       0.23      0.83      0.36        64
           1       0.98      0.77      0.86       761

    accuracy                           0.77       825
   macro avg       0.61      0.80      0.61       825
weighted avg       0.92      0.77      0.82       825

